In [29]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
import numpy as np
from getpass import getpass

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

### Conectar con la Collection en Mongo

In [30]:
def access_mdb_local_collection (database, collection):

    """ Function that returns a collection inside a DataBase 
    host in my local Server """

    # 1. Connect to my local Server
    client = MongoClient("localhost:27017")
    
    # 2. Access a DataBase:
    db = client[database]

    # 3. Acces a collection inside the given DataBase
    c = db.get_collection(collection)

    # 4. Return collection into a given DataBase host in a given Server
    return c

### EXTRACT

In [31]:
def extract_parameters(i):
            
            name = i["name"]
            
            lat = i["geocodes"]["main"]["latitude"]
            lon = i["geocodes"]["main"]["longitude"]
            
            #can have more than one:
            cat_list = [x["id"] for x in i["categories"]]
            
            try:
                city = i["location"]["locality"]
            except KeyError:
                city = np.nan
            
            params = {"city":city, "name": name, "cat_code":cat_list, "lat":lat, "lon":lon}
            
            return params

In [32]:
def heat_city(df, to_map):
    
    HeatMap(data = df[["lat", "lon"]]).add_to(to_map)
        
    return

# MAIN

In [51]:
c = access_mdb_local_collection ("ironhack", "foursquare")

# Uncluimos las tres condiciones para que los datos sean lo mas completo posibles
cond1 = {"name": {"$regex": "[Uu]niversidad"}}
cond2 = {"categories.id": 12125 } #--> mas restrictivo
cond3 = {"categories.id": 12013}

filt = {"$or": [cond1, cond2, cond3]}
    
# 2. Select the filds that we want to target:
projection = { "_id":0,"categories":1, "geocodes":1,"name":1, "location":1}
    
# 3. Extract data
datos = list(c.find(filt,projection))

# 4. Generate an empty dictionary with the categories to save:
register = {"city":[], "place_name":[], "categories":[],"lat":[], "lon":[]}

for i in datos:

        info = extract_parameters(i) # info it's a dictionary 
        
        register["city"].append(info["city"])
        register["place_name"].append(info["name"])
        register["categories"].append(info["cat_code"])
        register["lat"].append(info["lat"])
        register["lon"].append(info["lon"])

df = pd.DataFrame(register)


In [62]:
df.head(5)

,city,place_name,categories,lat,lon
0,Madrid,Estadio Nacional Universidad Complutense,[10051],40.436645,-3.732078
1,La Coruña,Facultade de Economía e Empresa,"[12013, 12125]",43.331011,-8.412856
2,Alacant,CECDL Centro de Estudios Ciudad de la Luz,"[12016, 12125]",38.310656,-0.526778
3,Barcelona,EAE Business School,"[12125, 12014]",41.380347,2.148989
4,Barcelona,Esic,"[12125, 12013]",41.396022,2.146088


In [58]:
spn_map = Map([40.40841191, -3.68760088], zoom_start=6, tiles='CartoDB positron')

In [59]:
heat_city(df, spn_map)

In [60]:
spn_map.save(f"spain_education_heatmap.html")

In [61]:
spn_map

In [ ]:
## Si te quieres motivar haz un barplot

# WINERS: Madrid, Valencia, Oviedo